In [3]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
import tensorflow as tf

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

from dask.distributed import Client, LocalCluster
# client.close()
cluster = LocalCluster()
client = Client(cluster)
print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
print(cluster,client)

/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44721 instead
  f"Port {expected} is already in use.\n"


LocalCluster(ab3e6dcd, 'tcp://127.0.0.1:35237', workers=4, threads=16, memory=24.59 GiB) <Client: 'tcp://127.0.0.1:35237' processes=4 threads=16, memory=24.59 GiB>
LocalCluster(ab3e6dcd, 'tcp://127.0.0.1:35237', workers=4, threads=16, memory=24.59 GiB) <Client: 'tcp://127.0.0.1:35237' processes=4 threads=16, memory=24.59 GiB>


In [2]:
client.close()

In [4]:
#making_label_set

# crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
# disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
#            '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
# risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


# #ensemble_labler
# label_description = {}
# for key, value in disease.items():
#     label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
#     for disease_code in value:
#         for risk_code in risk:
#             label = f'{key}_{disease_code}_{risk_code}'
#             label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

# global ensemble_label_encoder
# ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
# ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

# #crop_labler
# crop_label_description = {}
# for key, value in disease.items():
#     crop_label_description[f'{key}'] = f'{crop[key]}'
    
# global crop_label_encoder
# crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
# crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


# #disease_labler
# disease_label_description = {}
# for key, value in disease.items():
#     disease_label_description[f'00'] = f'{crop[key]}'
#     for disease_code in value:
#         label = f'{disease_code}'
#         disease_label_description[label] = f'{disease_code}'

# global disease_label_encoder
# disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
# disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


# #risk_labler
# risk_label_description = {}
# for key, value in risk.items():
#     label = f'{key}'
#     risk_label_description[label] = f'{key}'

# global risk_label_encoder
# risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
# risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [5]:
# @delayed
# def label_encoding(label):
#     global ensemble_label_encoder
#     encoded_label = ensemble_label_encoder[label]
#     return encoded_label

# @delayed
# def label_encoding_crop(label):
#     global crop_label_encoder
#     encoded_label = crop_label_encoder[label]
#     return encoded_label

# @delayed
# def label_encoding_disease(label):
#     global disease_label_encoder
#     encoded_label = disease_label_encoder[label]
#     return encoded_label

# @delayed
# def label_encoding_risk(label):
#     global risk_label_encoder
#     encoded_label = risk_label_encoder[label]
#     return encoded_label

# def getlable(jsonpath,type):
#     if type == "ensemble":
#         with open(jsonpath, 'r') as f:
#             json_file = json.load(f)

#         crop = json_file['annotations']['crop']
#         disease = json_file['annotations']['disease']
#         risk = json_file['annotations']['risk']
#         label = f'{crop}_{disease}_{risk}'
#         return label

#     elif type == "crop":
#         with open(jsonpath, 'r') as f:
#             json_file = json.load(f)
#         crop = json_file['annotations']['crop']
#         label = f'{crop}'
#         return label

#     elif type == "dc":
#         with open(jsonpath, 'r') as f:
#             json_file = json.load(f)
#         disease = json_file['annotations']['disease']
#         label = f'{disease}'
#         return label

#     elif type == "risk":  
#         with open(jsonpath, 'r') as f:
#             json_file = json.load(f)
#         risk = json_file['annotations']['risk']
#         label = f'{risk}'
#         return label

    
# def get_label_list(labelpath_list):
#     labelarr = np.array([])
#     labelarr_crop = np.array([])
#     labelarr_dc = np.array([])
#     labelarr_risk = np.array([])


#     # labelarr = da.array([])
#     for ind,json_path in enumerate(labelpath_list):
#         # label = label_encoding(getlable(json_path))
#         # label = da.array(np.array(label_encoding(getlable(json_path))))
#         label = np.array(label_encoding(getlable(json_path,"ensemble")))
#         labelarr = np.append(labelarr,label)

#         label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
#         labelarr_crop = np.append(labelarr_crop,label_crop)

#         label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
#         labelarr_dc = np.append(labelarr_dc,label_dc)

#         label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
#         labelarr_risk = np.append(labelarr_risk,label_risk)


#     return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [27]:
label_description = {
                    "1_00_0" : "딸기", 
                    "2_00_0" : "토마토",
                    "2_a5_2" : "토마토_흰가루병_중기",
                    "3_00_0" : "파프리카",
                    "3_a9_1" : "파프리카_흰가루병_초기",
                    "3_a9_2" : "파프리카_흰가루병_중기",
                    "3_a9_3" : "파프리카_흰가루병_말기",
                    "3_b3_1" : "파프리카_칼슘결핍_초기",
                    "3_b6_1" : "파프리카_다량원소결필(N)_초기",
                    "3_b7_1" : "파프리카_다량원소결필(P)_초기",
                    "3_b8_1" : "파프리카_다량원소결필(K)_초기",
                    "4_00_0" : "오이",
                    "5_00_0" : "고추",
                    "5_a7_2" : "고추_탄저병_중기",
                    "5_b6_1" : "고추_다량원소결필(N)_초기",
                    "5_b7_1" : "고추_다량원소결필(P)_초기",
                    "5_b8_1" : "고추_다량원소결필(K)_초기",
                    "6_00_0" : "시설포도",
                    "6_a11_1" : "시설포도_탄저병_초기",
                    "6_a11_2" : "시설포도_탄저병_중기",
                    "6_a12_1" : "시설포도_노균병_초기",
                    "6_a12_2" : "시설포도_노균병_중기",
                    "6_b4_1" : "시설포도_일소피해_초기",
                    "6_b4_3" : "시설포도_일소피해_말기",
                    "6_b5_1" : "시설포도_축과병_초기"   }


global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}
@delayed
def label_encoding(label):
    global label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

def getlable(jsonpath):
    with open(jsonpath, 'r') as f:
        json_file = json.load(f)

    crop = json_file['annotations']['crop']
    disease = json_file['annotations']['disease']
    risk = json_file['annotations']['risk']
    label = f'{crop}_{disease}_{risk}'
    return label
    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path)))
        labelarr = np.append(labelarr,label)
    return labelarr

In [6]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

* 데이터셋 준비

In [7]:
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)
#making_img_set
row_img = dask_image.imread.imread(path+"/train/*/*.jpg")
train = [imageresize(img) for img in row_img]
train_x = np.array(dask.compute(*train))



In [29]:
# label_list = sorted(glob(path+"/train/*/*.json"))
# # y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = get_label_list(label_list)
# # y_train = y_train.rechunk(5767)
# results = dask.compute(*y_train)
# label = np.array(results)
# # print(label.shape)
# results_crop = dask.compute(*y_train_crop)
# label_crop = np.array(results_crop)
# #
# results_dc = dask.compute(*y_train_dc)
# label_dc = np.array(results_dc)
# #
# results_risk = dask.compute(*y_train_risk)
# label_risk = np.array(results_risk)
# label.shape,label_crop.shape,label_dc.shape,label_risk.shape

((5767,), (5767,), (5767,), (5767,))

In [ ]:
label_list = sorted(glob(path+"/train/*/*.json"))
# y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
y_train = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
label.shape

## 모바일넷2 파인튜닝

In [30]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(25,activation = 'softmax'),
])



base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # loss=tf.keras.losses.CategoricalCrossentropy,
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 25)                32025     
                                                                 
Total params: 2,290,009
Trainable params: 32,025
Non-trainable params: 2,257,984
_________________________________________________________________


/home/gob/anaconda3/envs/dacon_parm_01/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [31]:
y_encoded = pd.get_dummies(label)
y_encoded = y_encoded.to_numpy()
X, X_test, y, y_test = train_test_split(train_x, y_encoded, test_size=0.2, shuffle= True,stratify=label)

In [32]:
history = model.fit(X,y,
                    epochs=2,
                    batch_size = 300,
                    validation_data=(X_test,y_test))



# automodel.fit(x_train[0], x_train[0], 
#                  batch_size=1000, 
#                  epochs=50,
#                 )

Epoch 1/2
16/16 [==============================] - 75s 5s/step - loss: 3.6135 - accuracy: 0.0761 - val_loss: 3.1685 - val_accuracy: 0.1256
Epoch 2/2
16/16 [==============================] - 73s 5s/step - loss: 3.0649 - accuracy: 0.1442 - val_loss: 2.7594 - val_accuracy: 0.1967


In [33]:
y_pred = model.predict(X_test)

In [34]:
y_pred.shape, y_test.shape

((1154, 25), (1154, 25))

In [35]:
y_pred_decode = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
y_test_decode = [np.argmax(y_pred[i]) for i in range(len(y_test))]

In [36]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test_decode, y_pred_decode))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test_decode, y_pred_decode, digits=3))

[[345   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  19   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 241   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   2   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   5   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   3   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   4   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 117   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0]
 [  0   0   0   0   0   0

In [37]:
answer = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_test_decode])
predss = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_pred_decode])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추_다량원소결필(K)_초기,고추_다량원소결필(N)_초기,고추_다량원소결필(P)_초기,고추_탄저병_중기,딸기,시설포도,시설포도_축과병_초기,시설포도_탄저병_초기,오이,토마토,토마토_흰가루병_중기,파프리카,파프리카_다량원소결필(K)_초기,파프리카_다량원소결필(N)_초기,파프리카_다량원소결필(P)_초기,파프리카_칼슘결핍_초기,파프리카_흰가루병_중기,파프리카_흰가루병_초기
answer,,,,,,,,,,,,,,,,,,
고추_다량원소결필(K)_초기,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
고추_다량원소결필(N)_초기,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
고추_다량원소결필(P)_초기,0,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
고추_탄저병_중기,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
딸기,0,0,0,0,345,0,0,0,0,0,0,0,0,0,0,0,0,0
시설포도,0,0,0,0,0,369,0,0,0,0,0,0,0,0,0,0,0,0
시설포도_축과병_초기,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0
시설포도_탄저병_초기,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0
오이,0,0,0,0,0,0,0,0,117,0,0,0,0,0,0,0,0,0


In [39]:
from sklearn.metrics import f1_score
f1_score(y_test_decode, y_pred_decode,average='macro')

1.0

테스트셋 예측

In [40]:
row_img_test = dask_image.imread.imread(path+"/test/*/*.jpg")
test = [imageresize(img) for img in row_img_test]
#test_x = np.array(dask.compute(*train))

In [50]:
submission = pd.read_csv(path+"/sample_submission.csv")

batch = 2000
data_length = len(test)
loop_num = int(np.floor(data_length/batch))

count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*batch
        end_p = (count+1)*batch

        temp = np.array(compute(test[start_p:end_p])[0])
        y_pred = model.predict(temp)
        y_pred_decode = [np.argmax(y_pred[i]) for i in range(len(y_pred))]

        submission.loc[start_p:end_p-1,'label'] = np.array([ensemble_label_decoder[int(val)] for val in y_pred_decode])
        
    else:
        print("lask")
        start_p = count*batch
        
        temp = np.array(compute(test[start_p:])[0])
        y_pred = model.predict(temp)
        y_pred_decode = [np.argmax(y_pred[i]) for i in range(len(y_pred))]

        submission.loc[start_p:,'label'] = np.array([ensemble_label_decoder[int(val)] for val in y_pred_decode])
        print("lask test")
            
    count += 1



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
lask
lask test


In [ ]:
submission.to_csv(path+'/submissions/gogo_submission_09.csv', index=False)